In [1]:
import os
import sys
import argparse
import pandas as pd
import numpy as np
from glob import glob


# Tone learning task

In [3]:
task_id = 'ToneLearning'

project_dir = os.path.abspath('/bgfs/bchandrasekaran/krs228/data/FLT/')
behav_dir   = os.path.join(project_dir, 'sourcedata', 'behav_files', 'CHA-IBR/')
bids_dir    = os.path.join(project_dir, 'data_bids_noIntendedFor')
#project_dir = os.path.join('/Users/krs228', 'data', 'FLT')
#behav_dir = os.path.join('/Users/krs228/','OneDrive - University of Pittsburgh/','CHA-IBR/')

# bids task names
bids_task_list = ['tonecat', 'stgrid']

#subject_list = ['FLT01', 'FLT04', 'FLT06', 'FLT07',  
                 # 
#                'FLT08', 'FLT09', 'FLT10', 'FLT11', 'FLT12', 'FLT13'] # 
subject_list = ['FLT02', 'FLT03', 'FLT05', ]

In [4]:
for subject_id in subject_list:
    print(subject_id)

    file_list = sorted(glob(behav_dir + '/*%s*/sub-%s*.csv'%(task_id, subject_id)))
    #file_list = [sorted(glob(behav_dir + '/*%s*/sub-%s*.csv'%(task_id, subject_id)))[0]]
    print('file list: ', file_list)
    
    # define initial BOLD acquisition time before task begins during silent gap
    first_acq = 2

FLT02
file list:  ['/bgfs/bchandrasekaran/krs228/data/FLT/sourcedata/behav_files/CHA-IBR/01-ToneLearning/sub-FLT02_tone_learning_16-tone_jitter-fb_fmri_2022_Mar_28_1901.csv', '/bgfs/bchandrasekaran/krs228/data/FLT/sourcedata/behav_files/CHA-IBR/01-ToneLearning/sub-FLT02_tone_learning_16-tone_jitter-fb_fmri_2022_Mar_28_1913.csv', '/bgfs/bchandrasekaran/krs228/data/FLT/sourcedata/behav_files/CHA-IBR/01-ToneLearning/sub-FLT02_tone_learning_16-tone_jitter-fb_fmri_2022_Mar_28_1922.csv', '/bgfs/bchandrasekaran/krs228/data/FLT/sourcedata/behav_files/CHA-IBR/01-ToneLearning/sub-FLT02_tone_learning_16-tone_jitter-fb_fmri_2022_Mar_28_1931.csv', '/bgfs/bchandrasekaran/krs228/data/FLT/sourcedata/behav_files/CHA-IBR/01-ToneLearning/sub-FLT02_tone_learning_16-tone_jitter-fb_fmri_2022_Mar_28_1941.csv', '/bgfs/bchandrasekaran/krs228/data/FLT/sourcedata/behav_files/CHA-IBR/01-ToneLearning/sub-FLT02_tone_learning_16-tone_jitter-fb_fmri_2022_Mar_28_1950.csv']
FLT03
file list:  ['/bgfs/bchandrasekaran/krs

In [14]:
    ''' ToneLearning task '''
    if 'ToneLearning' in task_id:
        # in this task, stimuli start 0.5 s after the silent gap starts
        stim_delay = 0.5

        # define the time before the first stimulus starts
        first_stim_delay = first_acq + stim_delay
        
        run_i = 1
        for rx, filename in enumerate(file_list):
            #try:
            print('converting ', filename)
            fpath = os.path.join(behav_dir, filename)
            df = pd.read_csv(fpath)

            # create a temp dataframe of only trials where sounds were presented
            trial_df = df[df.corrAns>0]

            if len(trial_df)<30:
                print('too few trials – incomplete run. Skipping')
            else:
                ''' Stimulus dataframe '''
                # set up stimulus dataframe
                stim_df = pd.DataFrame(columns=['onset', 
                                                'duration', 
                                                'trial_type',
                                                'stim_file'])

                # define onset time (relative to the first stimulus presentation)
                stim_df.onset = trial_df['sound_1.started'] - (trial_df['sound_1.started'].iloc[0]-first_stim_delay)

                # define duration
                # stim_df.duration = trial_df['sound_1.stopped'].astype(np.float16) - trial_df['sound_1.started'].astype(np.float16)
                stim_df.duration = 0.3

                # define stimulus type (based on sound file – HARDCODED)
                stim_df.trial_type = 'sound_'+trial_df.soundfile.str[8:14]
                '''
                stim_df.trial_type[trial_df.soundfile=='stimuli/di1-aN_48000Hz_pol2_S15filt.wav'] = 'di1-aN'
                '''

                # define stimulus soundfile
                stim_df.stim_file = trial_df.soundfile

                ''' Response dataframe '''
                # set up response dataframe
                resp_df = pd.DataFrame(columns=['onset', 
                                                'duration',
                                                'response_time', 
                                                'correct_key',
                                                'trial_type'])

                # define onset time (relative to the first stimulus presentation)
                resp_df.onset = trial_df['sound_1.started'] + trial_df['key_resp.rt']  - (trial_df['sound_1.started'].iloc[0]-first_stim_delay)

                # define duration (arbitrary)
                resp_df.duration = 0.5

                resp_df.response_time = trial_df['key_resp.rt']        
                resp_df.correct_key = trial_df['corrAns']
                resp_df.trial_type = 'resp_'+trial_df['key_resp.keys']

                ''' Feedback dataframe '''
                # set up feedback dataframe
                fb_df = pd.DataFrame(columns=['onset',
                                                'duration', 
                                                'trial_type'])        

                # define onset time (relative to the first stimulus presentation)
                fb_df.onset = trial_df['text_2.started'] - (trial_df['sound_1.started'].iloc[0]-first_stim_delay)

                # feedback is visible from the onset of text_2 to the onset of jitter_cross_post_fb
                fb_df.duration = trial_df['jitter_cross_post_fb.started'] - trial_df['text_2.started']

                # define feedback presented
                fb_df['trial_type'] = np.where(trial_df['key_resp.corr']==1, 'fb_correct', 
                                                (np.where(trial_df.corrAns==0, 'none', 'fb_wrong')))

                ''' combine all three dataframes '''
                bids_df = pd.concat([stim_df, resp_df, fb_df], 
                                    axis=0, join='outer', ignore_index=True)
                bids_df.sort_values(by=['onset'], ignore_index=True,
                                    inplace=True)

                # save to output path
                out_fpath = os.path.join(bids_dir,
                                         'sub-%s'%subject_id, 'func',
                                         'sub-%s_task-%s_run-%02d_events.tsv'%(subject_id, bids_task_list[0], run_i))

                bids_df.to_csv(out_fpath, sep='\t')
                print('saved output to ', out_fpath)
                run_i += 1
           # except:
           #     print('could not process this csv file')
           #     print(df.head)
           #     pass

FLT02
file list:  ['/bgfs/bchandrasekaran/krs228/data/FLT/sourcedata/behav_files/CHA-IBR/01-ToneLearning/sub-FLT02_tone_learning_16-tone_jitter-fb_fmri_2022_Mar_28_1901.csv', '/bgfs/bchandrasekaran/krs228/data/FLT/sourcedata/behav_files/CHA-IBR/01-ToneLearning/sub-FLT02_tone_learning_16-tone_jitter-fb_fmri_2022_Mar_28_1913.csv', '/bgfs/bchandrasekaran/krs228/data/FLT/sourcedata/behav_files/CHA-IBR/01-ToneLearning/sub-FLT02_tone_learning_16-tone_jitter-fb_fmri_2022_Mar_28_1922.csv', '/bgfs/bchandrasekaran/krs228/data/FLT/sourcedata/behav_files/CHA-IBR/01-ToneLearning/sub-FLT02_tone_learning_16-tone_jitter-fb_fmri_2022_Mar_28_1931.csv', '/bgfs/bchandrasekaran/krs228/data/FLT/sourcedata/behav_files/CHA-IBR/01-ToneLearning/sub-FLT02_tone_learning_16-tone_jitter-fb_fmri_2022_Mar_28_1941.csv', '/bgfs/bchandrasekaran/krs228/data/FLT/sourcedata/behav_files/CHA-IBR/01-ToneLearning/sub-FLT02_tone_learning_16-tone_jitter-fb_fmri_2022_Mar_28_1950.csv']
converting  /bgfs/bchandrasekaran/krs228/data

In [15]:
bids_df

,onset,duration,trial_type,stim_file,response_time,correct_key
0,2.500000,0.300000,sound_di1-iN,stimuli/di1-iN_48000Hz_pol2_S15filt.wav,NaN,NaN
1,3.622022,0.100000,resp_7,NaN,1.122022,7.0
2,6.079869,0.749823,fb_correct,NaN,NaN,NaN
3,8.494255,0.300000,sound_di4-hN,stimuli/di4-hN_48000Hz_pol2_S15filt.wav,NaN,NaN
4,9.211985,0.100000,resp_2,NaN,0.717730,2.0
...,...,...,...,...,...,...
139,351.160549,0.100000,resp_1,NaN,1.352762,1.0
140,353.188481,0.765435,fb_correct,NaN,NaN,NaN
141,355.819401,0.300000,sound_di3-iN,stimuli/di3-iN_48000Hz_pol2_S15filt.wav,NaN,NaN
142,357.060995,0.100000,resp_1,NaN,1.241594,1.0


In [16]:
bids_df.trial_type.str.split('_', expand=True)[0]

0      sound
1       resp
2         fb
3      sound
4       resp
       ...  
139     resp
140       fb
141    sound
142     resp
143       fb
Name: 0, Length: 144, dtype: object

# STgrid task

In [2]:
task_id = 'STgrid'
project_dir = os.path.abspath('/bgfs/bchandrasekaran/krs228/data/FLT/')
behav_dir   = os.path.join(project_dir, 'sourcedata', 'behav_files', 'CHA-IBR/')
bids_dir    = os.path.join(project_dir, 'data_bids')
#project_dir = os.path.join('/Users/krs228', 'data', 'FLT')
#behav_dir = os.path.join('/Users/krs228/','OneDrive - University of Pittsburgh/','CHA-IBR/')

# bids task names
bids_task_list = ['tonecat', 'stgrid']

subject_list = ['FLT02', 'FLT03', 'FLT04', 'FLT05', 'FLT06', 'FLT07', 'FLT08', 'FLT09', 
                'FLT10', 'FLT11', 'FLT12', 'FLT13', 'FLT14', 'FLT15', 'FLT17', 'FLT18', 'FLT19', 
                'FLT20', 'FLT21', 'FLT22', 'FLT23', 'FLT24', 'FLT25', 'FLT26', 'FLT28', 'FLT30', ]

#### Read in modulation rates

In [3]:
stim_mod_fpath = os.path.join(project_dir, 'sourcedata', '2022-2-8_Grid.txt')
mod_df = pd.read_csv(stim_mod_fpath, sep='\t', header=None)

mod_df.columns = ['temp_mod_rate', 'spect_mod_rate']
mod_df.index += 1 # 1-index

In [5]:
mod_df

,temp_mod_rate,spect_mod_rate
1,1.600000,0.16
2,6.066667,0.16
3,10.533333,0.16
4,15.000000,0.16
5,1.600000,0.94
6,6.066667,0.94
7,10.533333,0.94
8,15.000000,0.94
9,1.600000,1.72
10,6.066667,1.72


### block design

In [7]:
for subject_id in subject_list:
    print(subject_id)

    file_list = sorted(glob(behav_dir + '/*%s*/sub-%s*.csv'%(task_id, subject_id)))
    print('file list: ', file_list)

    # define initial BOLD acquisition time before task begins during silent gap
    first_acq = 2
    stim_delay = 0.4

    # define the time before the first stimulus starts
    first_stim_delay = first_acq + stim_delay

    for rx, filename in enumerate(file_list):
        print('converting ', filename)
        fpath = os.path.join(behav_dir, filename)
        df = pd.read_csv(fpath)

        if len(df) < 100:
            print('too few trials. skipping')
        else:
            # define output path
            out_fpath = os.path.join(bids_dir, 
                                     'sub-%s'%subject_id, 'func',
                                     'sub-%s_task-%s_run-%02d_events.tsv'%(subject_id, 
                                                                           bids_task_list[1], 
                                                                           rx+1))

            # set up dataframe
            bids_df = pd.DataFrame(columns=['onset', 'duration', 'trial_type', 
                                            'temp_mod_rate', 'spect_mod_rate',
                                            'response_time'])

            #bids_df.onset = df['sound_stimulus.started']-(df['sound_stimulus.started'][1]-first_stim_delay)
            #bids_df.duration[df['sound_stimulus.started']>0] = 1.0

            onset_list = []
            stim_list = []
            temp_list = []
            spect_list = []
            for sx, stim in enumerate(df.soundFile):
                if isinstance(stim, str) and 'S15' in stim:
                    if stim != df.soundFile[sx-1]:
                        stim_num = int(stim.split('_')[3])
                        temp_mod = mod_df.loc[stim_num]['temp_mod_rate']
                        spect_mod = mod_df.loc[stim_num]['spect_mod_rate']

                        onset = (df['sound_stimulus.started'][sx]-
                                 (df['sound_stimulus.started'][1]-first_stim_delay))
                        onset_list.append(onset)

                        stim_list.append('stim%02d'%stim_num)
                        temp_list.append(temp_mod)
                        spect_list.append(spect_mod)

            bids_df.onset = onset_list        
            bids_df.trial_type = stim_list
            bids_df.temp_mod_rate = temp_list
            bids_df.spect_mod_rate = spect_list
            bids_df.duration = 20 
            print(bids_df)

            # save to output path
            bids_df.to_csv(out_fpath, sep='\t')
            print('saved output to ', out_fpath)

FLT02
file list:  ['/bgfs/bchandrasekaran/krs228/data/FLT/sourcedata/behav_files/CHA-IBR/02-STgrid/sub-FLT02_ST_grid-4x4_task-vis_2022_Mar_28_2003.csv', '/bgfs/bchandrasekaran/krs228/data/FLT/sourcedata/behav_files/CHA-IBR/02-STgrid/sub-FLT02_ST_grid-4x4_task-vis_2022_Mar_28_2011.csv']
converting  /bgfs/bchandrasekaran/krs228/data/FLT/sourcedata/behav_files/CHA-IBR/02-STgrid/sub-FLT02_ST_grid-4x4_task-vis_2022_Mar_28_2003.csv
         onset  duration trial_type  temp_mod_rate  spect_mod_rate  \
0     2.400000        20     stim03      10.533333            0.16   
1    22.380905        20     stim07      10.533333            0.94   
2    42.412384        20     stim08      15.000000            0.94   
3   102.373049        20     stim09       1.600000            1.72   
4   142.418910        20     stim02       6.066667            0.16   
5   162.383336        20     stim12      15.000000            1.72   
6   182.364712        20     stim15      10.533333            2.50   
7   202.37

In [47]:
mod_df.loc[stim_num]

temp_mod_rate     15.0
spect_mod_rate     2.5
Name: 16, dtype: float64

#### original per-event

In [8]:
#for subject_id in subject_list:
print(subject_id)

file_list = sorted(glob(behav_dir + '/*%s*/sub-%s*.csv'%(task_id, subject_id)))
print('file list: ', file_list)

# define initial BOLD acquisition time before task begins during silent gap
first_acq = 2


if 'STgrid' in task_id:
    stim_delay = 0.4

    # define the time before the first stimulus starts
    first_stim_delay = first_acq + stim_delay

    for rx, filename in enumerate([file_list[0]]):
        print('converting ', filename)
        fpath = os.path.join(behav_dir, filename)
        df = pd.read_csv(fpath)

        if len(df) < 100:
            print('too few trials. skipping')
        else:
            # define output path
            out_fpath = os.path.join(bids_dir, 
                                     'sub-%s'%subject_id, 'func',
                                     'sub-%s_task-%s_run-%02d_events.tsv'%(subject_id, 
                                                                           bids_task_list[1], 
                                                                           rx+1))

            # set up dataframe
            bids_df = pd.DataFrame(columns=['onset', 'duration', 'trial_type', 
                                            'temp_mod_rate', 'spect_mod_rate',
                                            'response_time', 'stim_file'])

            bids_df.onset = df['sound_stimulus.started']-(df['sound_stimulus.started'][1]-first_stim_delay)
            bids_df.duration[df['sound_stimulus.started']>0] = 1.0

            #bids_df.trial_type[df['sound_stimulus.started'] > 0]   = 'sound'
            #bids_df.trial_type[df.soundFile == 'stimuli/null.wav'] = 'silent'
            for sx, stim in enumerate(df.soundFile):
                if isinstance(stim, str):
                    if 'S15' in stim:
                        bids_df.trial_type[sx] = '_'.join(stim.split('_')[2:4]) 
                    elif 'null' in stim:
                        bids_df.trial_type[sx] = 'null'

            # define response time (minus stim delay)
            bids_df.response_time = df['key_resp.rt'] - stim_delay

            bids_df.stim_file = df.soundFile

            # drop the first row if it's not a stimulus
            try:
                # is an error if index has been removed
                pd.isna(bids_df.stim_file[0]) 
                bids_df.drop(axis=0, index=0, inplace=True)
            except:
                pass

            print(bids_df)

            # save to output path
            #bids_df.to_csv(out_fpath, sep='\t')
            print('saved output to ', out_fpath)

FLT05
file list:  ['/bgfs/bchandrasekaran/krs228/data/FLT/sourcedata/behav_files/CHA-IBR/02-STgrid/sub-FLT05_ST_grid-4x4_task-vis_2022_Apr_20_1629.csv', '/bgfs/bchandrasekaran/krs228/data/FLT/sourcedata/behav_files/CHA-IBR/02-STgrid/sub-FLT05_ST_grid-4x4_task-vis_2022_Apr_20_1639.csv']
converting  /bgfs/bchandrasekaran/krs228/data/FLT/sourcedata/behav_files/CHA-IBR/02-STgrid/sub-FLT05_ST_grid-4x4_task-vis_2022_Apr_20_1629.csv
         onset duration trial_type  response_time  \
1     2.400000      1.0    stim_16            NaN   
2     6.380042      1.0    stim_16       0.259862   
3    10.375805      1.0    stim_16            NaN   
4    14.388586      1.0    stim_16            NaN   
5    18.385069      1.0    stim_16            NaN   
..         ...      ...        ...            ...   
121        NaN      NaN        NaN            NaN   
122        NaN      NaN        NaN            NaN   
123        NaN      NaN        NaN            NaN   
124        NaN      NaN        NaN      

/scratch/slurm-1826033/ipykernel_47321/1915119973.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bids_df.duration[df['sound_stimulus.started']>0] = 1.0
/scratch/slurm-1826033/ipykernel_47321/1915119973.py:44: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bids_df.trial_type[sx] = '_'.join(stim.split('_')[2:4])
/scratch/slurm-1826033/ipykernel_47321/1915119973.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bids_df.trial_type[

In [32]:
df['sound_stimulus.started']

0             NaN
1      212.264734
2      216.244776
3      220.240539
4      224.253320
          ...    
121           NaN
122           NaN
123           NaN
124           NaN
125           NaN
Name: sound_stimulus.started, Length: 126, dtype: float64

In [7]:
df.head(20)

,soundFile,visual_stim,blocks.thisRepN,blocks.thisTrialN,blocks.thisN,blocks.thisIndex,trials.thisRepN,trials.thisTrialN,trials.thisN,trials.thisIndex,...,fixation_cross_2.started,fixation_cross_2.stopped,participant,session,run,date,expName,psychopyVersion,frameRate,Unnamed: 39
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,sub-FLT05,ses-001,run-02,2022_Apr_20_1639,ST_grid-4x4_task-vis,2021.2.3,59.952218,NaN
1,stimuli/2022-2-8_GenTestGrid_stim_7_S15filt.wav,x,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0,...,NaN,NaN,sub-FLT05,ses-001,run-02,2022_Apr_20_1639,ST_grid-4x4_task-vis,2021.2.3,59.952218,NaN
2,stimuli/2022-2-8_GenTestGrid_stim_7_S15filt.wav,+,0.0,0.0,0.0,6.0,0.0,1.0,1.0,4.0,...,NaN,NaN,sub-FLT05,ses-001,run-02,2022_Apr_20_1639,ST_grid-4x4_task-vis,2021.2.3,59.952218,NaN
3,stimuli/2022-2-8_GenTestGrid_stim_7_S15filt.wav,+,0.0,0.0,0.0,6.0,0.0,2.0,2.0,2.0,...,NaN,NaN,sub-FLT05,ses-001,run-02,2022_Apr_20_1639,ST_grid-4x4_task-vis,2021.2.3,59.952218,NaN
4,stimuli/2022-2-8_GenTestGrid_stim_7_S15filt.wav,+,0.0,0.0,0.0,6.0,0.0,3.0,3.0,1.0,...,NaN,NaN,sub-FLT05,ses-001,run-02,2022_Apr_20_1639,ST_grid-4x4_task-vis,2021.2.3,59.952218,NaN
5,stimuli/2022-2-8_GenTestGrid_stim_7_S15filt.wav,+,0.0,0.0,0.0,6.0,0.0,4.0,4.0,3.0,...,NaN,NaN,sub-FLT05,ses-001,run-02,2022_Apr_20_1639,ST_grid-4x4_task-vis,2021.2.3,59.952218,NaN
6,stimuli/2022-2-8_GenTestGrid_stim_7_S15filt.wav,NaN,0.0,0.0,0.0,6.0,NaN,NaN,NaN,NaN,...,NaN,NaN,sub-FLT05,ses-001,run-02,2022_Apr_20_1639,ST_grid-4x4_task-vis,2021.2.3,59.952218,NaN
7,stimuli/2022-2-8_GenTestGrid_stim_16_S15filt.wav,+,0.0,1.0,1.0,15.0,0.0,0.0,0.0,1.0,...,NaN,NaN,sub-FLT05,ses-001,run-02,2022_Apr_20_1639,ST_grid-4x4_task-vis,2021.2.3,59.952218,NaN
8,stimuli/2022-2-8_GenTestGrid_stim_16_S15filt.wav,+,0.0,1.0,1.0,15.0,0.0,1.0,1.0,4.0,...,NaN,NaN,sub-FLT05,ses-001,run-02,2022_Apr_20_1639,ST_grid-4x4_task-vis,2021.2.3,59.952218,NaN
9,stimuli/2022-2-8_GenTestGrid_stim_16_S15filt.wav,x,0.0,1.0,1.0,15.0,0.0,2.0,2.0,0.0,...,NaN,NaN,sub-FLT05,ses-001,run-02,2022_Apr_20_1639,ST_grid-4x4_task-vis,2021.2.3,59.952218,NaN


In [15]:
bids_df.head(20)

,onset,duration,trial_type,response_time,stim_file
1,2.400000,1.0,null,NaN,stimuli/null.wav
2,6.396365,1.0,null,NaN,stimuli/null.wav
3,10.392615,1.0,null,NaN,stimuli/null.wav
4,14.405394,1.0,null,0.229535,stimuli/null.wav
5,18.401831,1.0,null,NaN,stimuli/null.wav
6,NaN,NaN,null,NaN,stimuli/null.wav
7,22.397909,1.0,stim_14,NaN,stimuli/2022-2-8_GenTestGrid_stim_14_S15filt.wav
8,26.394266,1.0,stim_14,0.210388,stimuli/2022-2-8_GenTestGrid_stim_14_S15filt.wav
9,30.390600,1.0,stim_14,NaN,stimuli/2022-2-8_GenTestGrid_stim_14_S15filt.wav
10,34.403484,1.0,stim_14,NaN,stimuli/2022-2-8_GenTestGrid_stim_14_S15filt.wav


In [36]:
first_df = bids_df.groupby('trial_type', sort=False, as_index=False).first()

first_df.drop(first_df[first_df.trial_type=='null'].index, inplace=True)

first_df.duration = 20

In [38]:
first_df.trial_type = 'sound'

In [39]:
first_df

,trial_type,onset,duration,response_time,stim_file
1,sound,22.397909,20,0.210388,stimuli/2022-2-8_GenTestGrid_stim_14_S15filt.wav
2,sound,62.394020,20,0.147137,stimuli/2022-2-8_GenTestGrid_stim_1_S15filt.wav
3,sound,82.391925,20,NaN,stimuli/2022-2-8_GenTestGrid_stim_2_S15filt.wav
4,sound,102.390007,20,0.555418,stimuli/2022-2-8_GenTestGrid_stim_3_S15filt.wav
5,sound,122.404412,20,-0.019619,stimuli/2022-2-8_GenTestGrid_stim_4_S15filt.wav
6,sound,142.402470,20,0.149665,stimuli/2022-2-8_GenTestGrid_stim_5_S15filt.wav
7,sound,162.400312,20,0.026889,stimuli/2022-2-8_GenTestGrid_stim_6_S15filt.wav
8,sound,182.398360,20,0.077583,stimuli/2022-2-8_GenTestGrid_stim_15_S15filt.wav
9,sound,202.396330,20,-0.216713,stimuli/2022-2-8_GenTestGrid_stim_8_S15filt.wav
10,sound,222.394116,20,-0.022231,stimuli/2022-2-8_GenTestGrid_stim_11_S15filt.wav
